In [9]:
import requests 
from models.api_models import *
from typing import Optional, Union
from tqdm import tqdm
import time
import joblib

In [2]:
GAMES_IDS = []
with open('input/evaluate_games.txt', 'rt') as file:
    GAMES_IDS = list(map(str.strip, file.readlines()))

In [3]:
__STEAM_API_KEY = None
with open('private/api_key.txt', 'rt') as file:
    __STEAM_API_KEY = file.readline()
    
__MATCH_DETAILS_URL = 'https://api.steampowered.com/IDOTA2Match_570/GetMatchDetails/V001/'

In [4]:
def get_match_details(match_id) -> Union[int, Game]:
    response = requests.get(__MATCH_DETAILS_URL,
                            params={'key': __STEAM_API_KEY, 'match_id': match_id},
                            timeout=2)
    match = -1
    if response.ok:
        match = _parse_match_details(response.json())
        if match is None:
            match = 0

    return match


def _parse_match_details(json_body: dict) -> Optional[Game]:
    result = None
    if 'error' not in json_body['result'].keys():
        game_dict = json_body['result']
        if game_dict['game_mode'] == 2:
            clear_game_dict = subset_of_dict(game_dict, Game.__slots__)
            result = Game(**clear_game_dict)
    return result

In [5]:
GAMES = []
for game_id in tqdm(GAMES_IDS):
    GAME = get_match_details(game_id)
    GAMES.append(GAME)
    time.sleep(1.0)
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:30<00:00,  1.51s/it]


In [7]:
PICK_BANS = [ game.picks_bans for game in GAMES]

In [12]:
joblib.dump(PICK_BANS, 'input/pro_pb.joblib')

['input/pro_pb.joblib']